In [1]:
import ee

ee.Initialize()

In [2]:
model = ee.Classifier.load("projects/ee-nwrc-geomatics/assets/cnwi/cnwi_model_122")

In [6]:
import payload

plyd = payload.Payload


In [9]:
from cnwi.cnwilib.image_collection import DataCubeProcessor, S1Processor, ALOSProcessor
# process the data cube

def image_processing(features, s1, dc) -> ee.Image:
    
    # Data Cube Processing
    dc_col = ee.ImageCollection(dc).filterBounds(features)
    dc_proc = DataCubeProcessor(dc_col).process().collection

    # S1 Processing
    s1_col = ee.ImageCollection(s1).filterBounds(features)
    s1_proc = S1Processor(s1_col).process().collection

    alos_proc = ALOSProcessor(
        start_date='2018',
        end_date='2021',
        region=features,
    ).process().collection
    
    # combine all the images
    return ee.Image.cat(dc_proc, s1_proc, alos_proc)

region = ee.FeatureCollection("projects/ee-nwrc-geomatics/assets/cnwi/regions_122")
image = image_processing(region, plyd.s1, plyd.s2)


In [10]:
prediction = image.classify(model)

In [11]:
import geemap

Map = geemap.Map()

Map.addLayer(prediction, {'min': 1, 'max': 8}, "CNWI Prediction")
Map.centerObject(region, 8)
Map

Map(center=[45.886020240278434, -63.42401428370772], controls=(WidgetControl(options=['position', 'transparent…